In [1]:
import pickle
from bs4 import BeautifulSoup
import psycopg2
import pandas as pd

In [2]:
file_name = 'data/scrape_tv_list_60_2'
url = 'https://www.amazon.com/Samsung-Electronics-UN65MU6300-65-Inch-Ultra/product-reviews/B06XGXDDJZ/ref=cm_cr_dp_d_show_all_top?ie=UTF8&reviewerType=all_reviews'

In [13]:
def store_reviews(file, id, url):
    conn = psycopg2.connect(dbname='product_reviews', user='postgres', password='', host='localhost')
    cursor = conn.cursor()
    file_name = f'data/scraped_data/{file}_{id}'
    documents = pickle.load(open(file_name,"rb"))
    print (len(documents))
    product_name = ''
    p_no = 0
    r_no = 0
    for document in documents:
        soup = BeautifulSoup(document,'html.parser')
        p_no += 1
        reviews = soup.select("div.review")
        product_name = soup.select("div.product-title")[0].text.replace('\"','').replace('\'','')
        brand = soup.select("div.product-by-line")[0].text
        for review in reviews:
            r_no += 1
            links = review.findAll('a')
            stars = links[0].text
            title = links[1].text.replace('\"','').replace('\'','')
            reviewer_name = links[2].text.replace('\"','').replace('\'','')
            size = links[3].text
            comments = links[4].text + links[5].text + links[6].text
            comments = comments.replace('\n','')
            review_text = review.select('span.review-text')[0].text
            vote_text = review.select('span.cr-vote-text')
            review_date = review.select('span.review-date')[0].text
            sql = '''insert into reviews (product_name, url, page_no, r_no, r_stars, r_date, r_name, r_title, r_text, r_comments) 
            values ('{}','{}',{},{},'{}','{}','{}','{}','{}','{}');'''.format(product_name.replace('\'',''), url, p_no, r_no, stars, review_date[3:], reviewer_name, title.replace('\'',''), review_text.replace('\'',''), comments.replace('\'',''))
            cursor.execute(sql)
            conn.commit()
    cursor.close()
    conn.close()

In [8]:
!pwd

/Users/bhaskarballapragada/Documents/Galvanize/product_review_sentiments


In [15]:
df = pd.read_csv('data/scrape_tv_list_60.csv')
for row in df.iterrows():
     store_reviews('scrape_tv_list_60',int(row[1][0]), row[1][3])

48
167
50
15
18
10
32
15
7
15
33
22
4
4
2
4
8
5
1
7
1
2
